<a href="https://colab.research.google.com/github/fonteleslrivka/Emergence-of-Causal-Order-from-a-Pre-Geometric-Substrate/blob/main/Pictures_code_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# RUN THIS CELL ALONE IN A NEW NOTEBOOK CELL (Colab / Jupyter)
# generate_figures_integrated.py  — integrated reproducible pipeline
# Rebeca Lemos — 2025
# ---------------------------------------------------------------------------

import os
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh

plt.style.use("seaborn-v0_8")

# -----------------------------
# configuration (edit if needed)
# -----------------------------
OUT_DIR = "paper/figures"
os.makedirs(OUT_DIR, exist_ok=True)

EPS = 1e-12

# safe defaults tuned for ~8GB RAM:
DEFAULTS = dict(N=512, beta=1.0, k_sparse=60, k_save=40, seeds=10)

# -----------------------------
# numerical utilities
# -----------------------------
def stable_softmax(evals, clip_min=-100, clip_max=100):
    """Stable softmax on a 1D numpy array (returns probabilities summing to 1)."""
    ev = np.asarray(evals, dtype=np.float64)
    ev = np.clip(ev, clip_min, clip_max)
    m = np.max(ev)
    z = np.exp(ev - m)
    Z = np.sum(z)
    if not np.isfinite(Z) or Z == 0:
        # fallback uniform
        lam = np.ones_like(ev) / ev.size
    else:
        lam = z / Z
    # guard against denormals
    lam = np.clip(lam, 1e-300, 1.0)
    lam /= lam.sum()
    return lam

def safe_eigsh_dense_or_sparse(mat, k=40, which='LA'):
    """
    Wrapper that accepts either dense numpy array or sparse matrix.
    Tries eigsh (Lanczos) where appropriate; falls back to eigh on small matrices.
    Returns eigenvalues (1D array).
    """
    n = mat.shape[0]
    # if small matrix, use dense eigh
    if n <= 800:
        try:
            w = eigh(mat, eigvals_only=True)
            return np.asarray(w)
        except Exception:
            pass
    # ensure csr for eigsh
    try:
        if not isinstance(mat, csr_matrix):
            mat_sp = csr_matrix(mat)
        else:
            mat_sp = mat
        # choose k safely
        k_try = min(max(6, min(k, n-2)), n-2)
        vals = eigsh(mat_sp, k=k_try, which=which, return_eigenvectors=False)
        return np.asarray(vals)
    except Exception as e:
        # fallback dense
        try:
            w = eigh(mat, eigvals_only=True)
            return np.asarray(w)
        except Exception as e2:
            raise RuntimeError("Eigen decomposition failed: " + str(e2))

# -----------------------------
# model building (points -> rho -> g)
# -----------------------------
def make_points(N, dim=4, seed=0):
    rng = np.random.default_rng(seed)
    X = rng.normal(0, 1, size=(N, dim))
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    X /= norms
    return X

def make_rho_g_from_points(X, beta=1.0, normalize=True):
    """
    Build rho and g from point coordinates.
    rho_ij = exp(-D_ij / beta)
    g = -log(rho + eps)
    returns (rho, g) as dense float32 arrays (rho in [0,1])
    """
    D = squareform(pdist(X, metric="euclidean")).astype(np.float32)
    rho = np.exp(-D / beta).astype(np.float32)
    if normalize:
        rho = rho / (np.max(rho) + EPS)
    g = -np.log(rho + EPS).astype(np.float32)
    return rho, g

# -----------------------------
# spectral helpers and stats
# -----------------------------
def top_abs_eigvals_of_g(g, k_pos=40, k_neg=40):
    """
    Return sorted descending by absolute value of concatenated top positive and negative modes.
    k_pos/k_neg refer to number of largest algebraic and smallest algebraic eigenvalues computed.
    """
    # try LA and SA separately for speed (works on symmetric matrices)
    n = g.shape[0]
    k_pos = min(k_pos, max(6, n-2))
    k_neg = min(k_neg, max(6, n-2))
    try:
        vals_pos = safe_eigsh_dense_or_sparse(g, k=k_pos, which='LA')
        vals_neg = safe_eigsh_dense_or_sparse(g, k=k_neg, which='SA')
        all_vals = np.concatenate([vals_pos, vals_neg])
    except Exception as e:
        # fallback to dense eigh (may be slow)
        w = eigh(g, eigvals_only=True)
        all_vals = np.asarray(w)
    abs_sorted = np.sort(np.abs(all_vals))[::-1]
    return abs_sorted

# -----------------------------
# entropy and triad score
# -----------------------------
def safe_entropy_from_eigs(evals):
    """Compute entropy using stable softmax (interpreting evals as 'log-weights')."""
    lam = stable_softmax(evals)
    with np.errstate(divide='ignore', invalid='ignore'):
        S = -np.sum(lam * np.log(lam + 1e-300))
    if not np.isfinite(S):
        S = -np.sum(lam * np.log(np.clip(lam, 1e-300, None)))
    return float(S), lam

def triad_gap_and_isotropy(eigvals_abs):
    """
    Given abs-sorted eigenvalues (descending), compute triad gap ratio λ_mean(triad)/mean(rest)
    and isotropy metric (ratio between min and max of the triad).
    """
    if eigvals_abs.size < 6:
        return np.nan, np.nan
    triad = eigvals_abs[:3]
    rest_mean = np.mean(eigvals_abs[3:]) if eigvals_abs.size>3 else np.nan
    triad_mean = np.mean(triad)
    gap = triad_mean / (rest_mean + EPS)
    isotropy = np.min(triad) / (np.max(triad) + EPS)
    return float(gap), float(isotropy)

# -----------------------------
# projected Hessian (stability test)
# -----------------------------
def projected_hessian_fd(g0, m=10, h_rel=1e-5, include_cross=True, F_func=None):
    """
    Projected finite-difference Hessian in subspace spanned by top-m eigenvectors of g0.
    F_func should accept a matrix G and return scalar (objective, e.g., entropy).
    Returns H_proj (m x m), and also returns the top eigenvectors used.
    """
    n = g0.shape[0]
    if F_func is None:
        # default: entropy of rho ~ exp(-g) normalized
        def F_func_default(G):
            # compute eigenvalues of -G safely (dense)
            w = eigh(-G, eigvals_only=True)
            lam = stable_softmax(w)
            with np.errstate(divide='ignore', invalid='ignore'):
                S = -np.sum(lam * np.log(lam + 1e-300))
            return float(S)
        F = F_func_default
    else:
        F = F_func

    # dense diagonalization for evecs (we need top directions)
    evals, evecs = eigh(g0)
    idx = np.argsort(np.abs(evals))[::-1][:m]
    V = evecs[:, idx]    # shape (n, m)

    # build rank-1 perturbation matrices normalized by Frobenius norm
    Xs = []
    for i in range(m):
        Xi = np.outer(V[:,i], V[:,i])
        nf = np.linalg.norm(Xi, ord='fro')
        if nf == 0:
            Xs.append(Xi)
        else:
            Xs.append(Xi / nf)

    # compute F0
    F0 = F(g0)

    H = np.zeros((m, m), dtype=float)
    # diagonal terms
    for a in range(m):
        Xi = Xs[a]
        Hp = F(g0 + h_rel * Xi)
        Hm = F(g0 - h_rel * Xi)
        H[a,a] = (Hp - 2*F0 + Hm) / (h_rel*h_rel)
    # optional cross terms (symmetric)
    if include_cross:
        for a in range(m):
            for b in range(a+1, m):
                Xa = Xs[a]; Xb = Xs[b]
                Fpp = F(g0 + h_rel*Xa + h_rel*Xb)
                Fpm = F(g0 + h_rel*Xa - h_rel*Xb)
                Fmp = F(g0 - h_rel*Xa + h_rel*Xb)
                Fmm = F(g0 - h_rel*Xa - h_rel*Xb)
                H[a,b] = (Fpp - Fpm - Fmp + Fmm) / (4*h_rel*h_rel)
                H[b,a] = H[a,b]
    return H, V

# -----------------------------
# plotting helpers
# -----------------------------
def save_png(fig, name):
    path = os.path.join(OUT_DIR, name)
    fig.tight_layout()
    fig.savefig(path, dpi=150)
    print("Saved:", path)

# -----------------------------
# high-level figure routines
# -----------------------------
def fig_spectrum_log(N=512, seed=0, k=60):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X, beta=DEFAULTS['beta'])
    eigs = top_abs_eigvals_of_g(g, k_pos=k, k_neg=k)
    fig, ax = plt.subplots(figsize=(8,4))
    ax.semilogy(eigs[:min(len(eigs), 80)], 'o-', ms=4)
    ax.axvline(3.5, color='red', ls='--', label='1+3 region')
    ax.set(title=f"Log-scale spectrum of g (N={N})", xlabel="mode index", ylabel="|λ|")
    ax.grid(alpha=0.25)
    ax.legend()
    save_png(fig, f"fig_spectrum_log{N}.png")
    plt.close(fig)
    return eigs

def fig_boxplot_top12(N=512, seeds=20):
    top_vals = []
    for s in range(seeds):
        X = make_points(N, seed=s)
        rho, g = make_rho_g_from_points(X)
        eigs = top_abs_eigvals_of_g(g, k_pos=12, k_neg=12)[:12]
        top_vals.append(eigs)
    top_vals = np.array(top_vals)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.boxplot([top_vals[:,i] for i in range(12)], showfliers=True)
    ax.set_yscale('log')
    ax.set(title="Seed-to-seed variability of top 12 eigenvalues",
           xlabel="eigenvalue index", ylabel="|λ| (log)")
    ax.grid(alpha=0.25)
    save_png(fig, "fig_boxplot_top12.png")
    plt.close(fig)
    return top_vals

def fig_softmax_weights(N=512, seed=0, k=40):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    # get top eigenvalues (dense fallback if necessary)
    try:
        vals_pos = safe_eigsh_dense_or_sparse(g, k=k, which='LA')
        vals_neg = safe_eigsh_dense_or_sparse(g, k=k, which='SA')
        eigs = np.sort(np.abs(np.concatenate([vals_pos, vals_neg])))[::-1]
    except Exception:
        eigs = np.sort(np.abs(eigh(g, eigvals_only=True)))[::-1]
    # compute softmax weights on the top eigenvalues (we expect one weight ~1)
    weights = stable_softmax(eigs[:20])
    # ensure index 0 is visible (bar plot is clearer)
    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(np.arange(len(weights)), weights)
    ax.set(title="Softmax-stable eigenvalue weights (top 20)",
           xlabel="index", ylabel="weight")
    ax.set_ylim(0, 1.05)
    save_png(fig, "fig_softmax_weights.png")
    plt.close(fig)
    return weights

def fig_hessian_eigs(N=512, seed=0, m=6):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    Hproj, V = projected_hessian_fd(g, m=m, h_rel=1e-5, include_cross=True)
    w = np.linalg.eigvalsh(Hproj)
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(w, 'o-')
    ax.set(title="Projected Hessian eigenvalues (projected subspace)", xlabel="index", ylabel="evalue")
    ax.grid(alpha=0.25)
    save_png(fig, "fig_hessian_eigs.png")
    plt.close(fig)
    return w, Hproj, V

def fig_overlap_zero(N=512, seed=0, m=6):
    X = make_points(N, seed=seed)
    rho, g = make_rho_g_from_points(X)
    evals, evecs = eigh(g)
    idx = np.argsort(np.abs(evals))[::-1][:m]
    V = evecs[:, idx]
    Hproj, Vproj = projected_hessian_fd(g, m=m, h_rel=1e-5, include_cross=True)
    w, U = np.linalg.eig(Hproj)
    idx_big = np.argmax(np.abs(w))
    unstable_vec = U[:, idx_big]
    v_rec = V @ unstable_vec
    v0 = V[:, 0]
    overlap = abs(np.dot(v_rec, v0)) / (np.linalg.norm(v_rec) * np.linalg.norm(v0) + 1e-30)
    fig, ax = plt.subplots(figsize=(6,4))
    ax.bar(['overlap'], [overlap])
    ax.set_ylim(0,1)
    ax.set(title="Overlap between Hessian extremal mode and dominant mode")
    save_png(fig, "fig_overlap_zero.png")
    plt.close(fig)
    return overlap

def fig_mean_spectrum(Ns=(512,1024), seeds=6):
    curves = []
    for N in Ns:
        vals_all = []
        for s in range(seeds):
            X = make_points(N, seed=s)
            rho, g = make_rho_g_from_points(X)
            vals_pos = safe_eigsh_dense_or_sparse(g, k=40, which='LA')
            vals = np.sort(np.abs(vals_pos))[::-1]
            vals_all.append(vals[:40])
        curves.append(np.mean(vals_all, axis=0))
    fig, ax = plt.subplots(figsize=(8,5))
    for N, curve in zip(Ns, curves):
        ax.semilogy(curve, 'o-', label=f"N={N}")
    ax.legend(); ax.grid(alpha=0.25)
    ax.set(title="Mean spectral curves", xlabel="index", ylabel="|λ|")
    save_png(fig, "fig_mean_spectrum_Ns.png")
    plt.close(fig)
    return curves

def fig_gap_hist(N=512, seeds=20):
    gaps = []
    for s in range(seeds):
        X = make_points(N, seed=s)
        rho, g = make_rho_g_from_points(X)
        vals = top_abs_eigvals_of_g(g, k_pos=10, k_neg=10)
        if vals.size >= 5:
            gaps.append(float(vals[3] / (vals[4] + EPS)))
    fig, ax = plt.subplots(figsize=(6,4))
    ax.hist(gaps, bins=12)
    ax.set(title="Distribution of gap ratio λ4/λ5", xlabel="λ4/λ5", ylabel="count")
    save_png(fig, "fig_gap_hist.png")
    plt.close(fig)
    return gaps

def fig_conceptual_diagrams():
    # causal condensation
    fig, ax = plt.subplots(figsize=(7,4))
    x = np.linspace(-3,3,300)
    y = np.exp(-x*x)
    ax.plot(x,y)
    ax.set(title="Conceptual Diagram: Causal Condensation")
    save_png(fig, "fig_causal_condensation.png")
    plt.close(fig)
    # pregeo vs geo
    fig, ax = plt.subplots(figsize=(7,4))
    ax.text(0.1,0.6,"Pre-geometric phase:\nOne axis only\nNo spatial pattern", fontsize=12)
    ax.text(0.6,0.6,"Geometric phase (contrast):\nMultiple axes structured", fontsize=12)
    ax.axis('off')
    save_png(fig, "fig_pregeo_vs_geo.png")
    plt.close(fig)

# -----------------------------
# orchestrator: run everything (safe defaults)
# -----------------------------
def run_all_figures(N=DEFAULTS['N'], seeds=DEFAULTS['seeds'], beta=DEFAULTS['beta'], k=DEFAULTS['k_sparse']):
    t0 = time.time()
    print("Running integrated figure generation...")
    print(f"N={N}, seeds={seeds}, beta={beta}, k={k}")
    # main figures
    eigs_512 = fig_spectrum_log(N=N, seed=0, k=k)
    topvals = fig_boxplot_top12(N=N, seeds=min(seeds,20))
    weights = fig_softmax_weights(N=N, seed=0, k=40)
    w_hess, Hproj, V = fig_hessian_eigs(N=N, seed=0, m=6)
    overlap = fig_overlap_zero(N=N, seed=0, m=6)
    curves = fig_mean_spectrum(Ns=(N, min(N*2, 2048)), seeds=min(6, seeds))
    gaps = fig_gap_hist(N=N, seeds=min(20,seeds))
    fig_conceptual_diagrams()

    # save summary npz (top eigs from last run)
    summary = dict(N=N, seeds=seeds, beta=beta, k=k,
                   eigs_sample=eigs_512[:min(200, eigs_512.size)].tolist(),
                   mean_top12=np.mean(topvals, axis=0).tolist(),
                   softmax_weights=weights.tolist(),
                   hess_eigs=w_hess.tolist(),
                   overlap=overlap,
                   gap_stats=dict(mean_gap=np.mean(gaps) if len(gaps)>0 else np.nan,
                                  median_gap=np.median(gaps) if len(gaps)>0 else np.nan))
    np.savez_compressed(os.path.join(OUT_DIR, f"figs_summary_N{N}.npz"), **summary)
    t1 = time.time()
    print(f"All done in {t1-t0:.1f}s. Figures saved to {OUT_DIR}")
    return summary

# -----------------------------
# if run as main (execute full pipeline)
# -----------------------------
if __name__ == "__main__":
    # safe run with defaults (change by editing the call)
    run_all_figures(N=DEFAULTS['N'], seeds=DEFAULTS['seeds'], beta=DEFAULTS['beta'], k=DEFAULTS['k_sparse'])

# ---------------------------------------------------------------------------
# End of integrated pipeline cell.
# Notes:
# - If ARPACK fails on eigsh for some seeds, the wrapper falls back to dense methods.
# - For larger N (>=2048) reduce k and seeds or run on a machine with more RAM.
# - If you want only subset of figures, call the specific functions above after importing.
# ---------------------------------------------------------------------------


Running integrated figure generation...
N=512, seeds=10, beta=1.0, k=60
Saved: paper/figures/fig_spectrum_log512.png
Saved: paper/figures/fig_boxplot_top12.png
Saved: paper/figures/fig_softmax_weights.png
Saved: paper/figures/fig_hessian_eigs.png
Saved: paper/figures/fig_overlap_zero.png
Saved: paper/figures/fig_mean_spectrum_Ns.png
Saved: paper/figures/fig_gap_hist.png
Saved: paper/figures/fig_causal_condensation.png
Saved: paper/figures/fig_pregeo_vs_geo.png
All done in 599.6s. Figures saved to paper/figures
